In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML

In [2]:
def show_scrollable(df, max_rows=20, height=400):
    html = df.head(max_rows).to_html(notebook=True)
    display(HTML(f'<div style="overflow:auto; height:{height}px">{html}</div>'))

In [3]:
after_Ture = pd.read_csv('Data\EfterkodTur_Nicola.csv', encoding='latin1')
df_Ture = pd.read_csv('Data\TuTure_Nicola.csv', encoding='latin1')

In [4]:
df_Ture

,turid,sessionid,turnr,tiladrflag,tiladrsogeord,tiladrid,tiladrstedid,tiladrtext,tiladrn,tiladre,tilkomnr,formaal
0,2118077,347009,1,4,Ahornvej 30 2970 Hørshol,2230102030_,0.0,"Ahornvej 30, 2970 Hørsholm",6199137,718433,223.0,23.0
1,2118078,347009,2,1,NaN,2230548038_,0.0,"Bellisvej 38, 2970 Hørsholm",6198520,718307,223.0,1.0
2,2118081,347010,3,4,Høsterkøb kirke,2300226286_,166952.0,"HØSTERKØB KIRKE, Gøngehusvej 286, 2970 Hørsholm",6195042,718391,230.0,43.0
3,2118082,347010,4,1,NaN,2300323083_,0.0,"Keilstruplund 83, 3460 Birkerød",6193548,715687,230.0,1.0
4,2118083,347013,1,4,Gevninge Bygade 2 4000,3500224002_,158249.0,"Dagli´Brugsen, Gevninge Bygade 2, Gevninge, 40...",6170097,686374,350.0,31.0
...,...,...,...,...,...,...,...,...,...,...,...,...
233519,2747013,538780,2,4,"Havnen 27, 4600 Køge",2593477027_,0.0,"Havnen 27, 4600 Køge",6150077,701552,259.0,51.0
233520,2747014,538780,3,1,NaN,1015044005_,0.0,"Nordre Frihavnsgade 5, 2100 København Ø",6178508,724839,101.0,1.0
233521,2747056,538797,2,1,NaN,1470475110_,0.0,"Kong Georgs Vej 110, 2000 Frederiksberg",6177225,721865,147.0,1.0
233522,2747120,538818,1,6,NaN,4615714001_,0.0,"Niels Bohrs Allé 1, 5230 Odense M",6137182,588694,461.0,11.0


In [5]:
cols_to_int = ['tiladrstedid','tilkomnr', 'formaal' ]
for col in cols_to_int:
    if col in df_Ture.columns:
        df_Ture[col] = pd.to_numeric(df_Ture[col], errors='coerce').astype('Int64')

In [6]:
cols_to_int = ['tiladre', 'tiladrn', 'destmuncode', 'tiladrstedid', 'turnr']
for col in cols_to_int:
    if col in after_Ture.columns:
        after_Ture[col] = pd.to_numeric(after_Ture[col], errors='coerce').astype('Int64')

In [7]:
after_Ture.rename(columns={'destmuncode':'tilkomnr'}, inplace=True)

In [8]:
# Identify common columns
common_cols = sorted(set(df_Ture.columns).intersection(after_Ture.columns))
compare_cols = [c for c in common_cols if c != "turid"]  # skip merge key

# Assign likely numeric vs text 
numeric_cols = [c for c in compare_cols if c in ["turnr","tiladrid","tiladrstedid","tiladrn", "tiladre"]]
text_cols    = [c for c in compare_cols if c in ["tiladrtext"]]

# Any leftover goes to text
for c in compare_cols:
    if c not in numeric_cols and c not in text_cols:
        text_cols.append(c)

print("Compare cols:", compare_cols)
print("Numeric cols:", numeric_cols)
print("Text cols:", text_cols)

Compare cols: ['tiladre', 'tiladrid', 'tiladrn', 'tiladrstedid', 'tiladrtext', 'tilkomnr', 'turnr']
Numeric cols: ['tiladre', 'tiladrid', 'tiladrn', 'tiladrstedid', 'turnr']
Text cols: ['tiladrtext', 'tilkomnr']


In [9]:
# Merge on turid
m = df_Ture.merge(after_Ture, on="turid", suffixes=("_raw","_mod"), how="inner")
print("Merged shape:", m.shape)

# Normalize numeric columns to nullable Int64
for c in numeric_cols:
    m[f"{c}_raw"] = pd.to_numeric(m[f"{c}_raw"], errors="coerce").astype("Int64")
    m[f"{c}_mod"] = pd.to_numeric(m[f"{c}_mod"], errors="coerce").astype("Int64")

# Normalize text columns (strip, lower, keep NA as NA)
for c in text_cols:
    m[f"{c}_raw"] = m[f"{c}_raw"].where(m[f"{c}_raw"].notna(), pd.NA).astype("string").str.strip().str.lower()
    m[f"{c}_mod"] = m[f"{c}_mod"].where(m[f"{c}_mod"].notna(), pd.NA).astype("string").str.strip().str.lower()

Merged shape: (233524, 21)


In [10]:
diff_masks = {}
for c in compare_cols:
    left  = m[f"{c}_raw"]
    right = m[f"{c}_mod"]
    # "changed" means values are different AND modified is not NA
    changed = right.notna() & (~right.eq(left))
    diff_masks[c] = changed

# Trip-level flag
any_changed = pd.DataFrame(diff_masks).any(axis=1)
m["flag"] = m["turid"].where(any_changed, "OK").mask(any_changed, "WRONG")

print("Total trips:", len(m))
print("Corrected trips:", (m["flag"]=="WRONG").sum())
print("% corrected:", (m["flag"]=="WRONG").mean()*100)
show_scrollable(m, max_rows=30)

Total trips: 233524
Corrected trips: 28327
% corrected: 12.130230725749817


,turid,sessionid,turnr_raw,tiladrflag,tiladrsogeord,tiladrid_raw,tiladrstedid_raw,tiladrtext_raw,tiladrn_raw,tiladre_raw,tilkomnr_raw,formaal,medtagtur,turnr_mod,tiladrid_mod,tiladrstedid_mod,tiladrtext_mod,tiladrn_mod,tiladre_mod,tilkomnr_mod,tiladrkonklu,flag
0,2118077,347009,1,4,Ahornvej 30 2970 Hørshol,<NA>,0,"ahornvej 30, 2970 hørsholm",6199137,718433,223,23,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,OK
1,2118078,347009,2,1,NaN,<NA>,0,"bellisvej 38, 2970 hørsholm",6198520,718307,223,1,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,OK
2,2118081,347010,3,4,Høsterkøb kirke,<NA>,166952,"høsterkøb kirke, gøngehusvej 286, 2970 hørsholm",6195042,718391,230,43,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,OK
3,2118082,347010,4,1,NaN,<NA>,0,"keilstruplund 83, 3460 birkerød",6193548,715687,230,1,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,OK
4,2118083,347013,1,4,Gevninge Bygade 2 4000,<NA>,158249,"dagli´brugsen, gevninge bygade 2, gevninge, 40...",6170097,686374,350,31,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,OK
5,2118084,347013,2,1,NaN,<NA>,0,"herslevvej 12, trællerup, 4000 roskilde",6172090,686952,350,1,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,OK
6,2118087,347015,1,4,Dalgas allé 7800,<NA>,221795,"fakta 0151, dalgas alle 18, 7800 skive",6267816,500884,779,11,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,OK
7,2118088,347015,2,1,NaN,<NA>,0,"vejlgårdvej 26, rønbjerg, 7800 skive",6266008,495786,779,1,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,OK
8,2118090,347016,1,2,NaN,<NA>,0,"lærkegårdsvænget 5, ordrup, 4540 fårevejle",6189478,651675,306,1,0,<NA>,<NA>,<NA>,<NA>,6189479,651680,<NA>,0,WRONG
9,2118092,347017,1,1,NaN,<NA>,0,"hirsevej 17, 5800 nyborg",6132795,614111,450,1,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,OK


In [11]:
m.columns

Index(['turid', 'sessionid', 'turnr_raw', 'tiladrflag', 'tiladrsogeord',
       'tiladrid_raw', 'tiladrstedid_raw', 'tiladrtext_raw', 'tiladrn_raw',
       'tiladre_raw', 'tilkomnr_raw', 'formaal', 'medtagtur', 'turnr_mod',
       'tiladrid_mod', 'tiladrstedid_mod', 'tiladrtext_mod', 'tiladrn_mod',
       'tiladre_mod', 'tilkomnr_mod', 'tiladrkonklu', 'flag'],
      dtype='object')

In [12]:
df_t = m.copy()  # Use merged dataframe for further analysis
df_t.drop(columns=['turnr_raw', 'tiladrflag',
       'tiladrid_raw', 'tiladrstedid_raw', 'tilkomnr_raw', 'formaal', 'medtagtur', 'turnr_mod',
       'tiladrid_mod', 'tiladrstedid_mod', 'tilkomnr_mod', 'tiladrkonklu', 'flag'], inplace=True)

In [13]:
# Build a boolean DataFrame: rows=trips, cols=fields, value=True if that field changed
diff_df = pd.DataFrame(diff_masks)  # same index as m

# List of changed columns per row - handle NA values by replacing them with False
m["changed_cols"] = diff_df.apply(lambda r: [c for c, v in r.items() if pd.notna(v) and v], axis=1)

In [14]:
tur_edit = m[['turid', 'sessionid', 'tiladrsogeord', 'tiladrtext_raw', 'flag', 'changed_cols']]

In [15]:
df_t

,turid,sessionid,tiladrsogeord,tiladrtext_raw,tiladrn_raw,tiladre_raw,tiladrtext_mod,tiladrn_mod,tiladre_mod
0,2118077,347009,Ahornvej 30 2970 Hørshol,"ahornvej 30, 2970 hørsholm",6199137,718433,<NA>,<NA>,<NA>
1,2118078,347009,NaN,"bellisvej 38, 2970 hørsholm",6198520,718307,<NA>,<NA>,<NA>
2,2118081,347010,Høsterkøb kirke,"høsterkøb kirke, gøngehusvej 286, 2970 hørsholm",6195042,718391,<NA>,<NA>,<NA>
3,2118082,347010,NaN,"keilstruplund 83, 3460 birkerød",6193548,715687,<NA>,<NA>,<NA>
4,2118083,347013,Gevninge Bygade 2 4000,"dagli´brugsen, gevninge bygade 2, gevninge, 40...",6170097,686374,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...
233519,2747013,538780,"Havnen 27, 4600 Køge","havnen 27, 4600 køge",6150077,701552,<NA>,<NA>,<NA>
233520,2747014,538780,NaN,"nordre frihavnsgade 5, 2100 københavn ø",6178508,724839,<NA>,<NA>,<NA>
233521,2747056,538797,NaN,"kong georgs vej 110, 2000 frederiksberg",6177225,721865,<NA>,<NA>,<NA>
233522,2747120,538818,NaN,"niels bohrs allé 1, 5230 odense m",6137182,588694,<NA>,<NA>,<NA>


In [16]:
df_t.to_csv('Data/EfterkodTur_Edited_Nicola.csv', index=False, encoding='latin1')

In [17]:
# tur_edit.to_csv('Data/TuTur_Edited_Nicola.csv', index=False, encoding='latin1')

## From here we can check and analyze the processed data, with the retriever

On the file 'process_csv' we use as upload file 'TuTur_Edited_Nicola' , process it and save it as 'TuTur_Test'

In [18]:
df_tur_test = pd.read_csv('Data/TuTur_Test.csv', encoding='latin1')

In [19]:
df_tur_test = df_tur_test.iloc[:, [0,1,2,3,6,7,8,9,10,11,4,5]]
df_tur_test['% opt 1'] = df_tur_test['% opt 1'].round().astype('Int64')
df_tur_test['% opt 2'] = df_tur_test['% opt 2'].round().astype('Int64')
df_tur_test['% opt 3'] = df_tur_test['% opt 3'].round().astype('Int64')

In [20]:
show_scrollable(df_tur_test, max_rows=50)

,turid,sessionid,tiladrsogeord,tiladrtext_raw,option 1,% opt 1,option 2,% opt 2,option 3,% opt 3,flag,changed_cols
0,2118077,347009,Ahornvej 30 2970 HÃ¸rshol,"ahornvej 30, 2970 hÃ¸rsholm","Ahornvej 30, 2970 HÃ¸rsholm",100,"Ahornvej 30, 2791 DragÃ¸r",80,"Ahornvej 30, 3630 JÃ¦gerspris",80,OK,[]
1,2118078,347009,NaN,"bellisvej 38, 2970 hÃ¸rsholm","Bellisvej 38, 2970 HÃ¸rsholm",100,"Bellisvej 38, 4000 Roskilde",80,"Bellisvej 38, 5000 Odense C",80,OK,[]
2,2118081,347010,HÃ¸sterkÃ¸b kirke,"hÃ¸sterkÃ¸b kirke, gÃ¸ngehusvej 286, 2970 hÃ¸r...","GÃ¸ngehusvej 286, 2970 HÃ¸rsholm",100,"GÃ¸ngehusvej 38, 2950 VedbÃ¦k",70,"Husvej 11, 7884 Fur",65,OK,[]
3,2118082,347010,NaN,"keilstruplund 83, 3460 birkerÃ¸d","Keilstruplund 83, 3460 BirkerÃ¸d",100,"Kelstruplund 1, 4560 Vig",65,"Lystruplund 78, 8520 Lystrup",65,OK,[]
4,2118083,347013,Gevninge Bygade 2 4000,"dagliÂ´brugsen, gevninge bygade 2, gevninge, 4...","Gevninge Bygade 7B, 4000 Roskilde",90,"Bygade 2, 7160 TÃ¸rring",65,"Genner Bygade 2, 6230 RÃ¸dekro",65,OK,[]
5,2118084,347013,NaN,"herslevvej 12, trÃ¦llerup, 4000 roskilde","Herslevvej 12, 4000 Roskilde",100,"Herslevvej 12, 5900 RudkÃ¸bing",80,"Herslevvejen 12, 4270 HÃ¸ng",65,OK,[]
6,2118087,347015,Dalgas allÃ© 7800,"fakta 0151, dalgas alle 18, 7800 skive","Dalgas Alle 18, 7800 Skive",100,"BrÃ¸nshÃ¸jholms AllÃ© 18, 2700 BrÃ¸nshÃ¸j",65,"Amalie Skrams AllÃ© 18, 2500 Valby",65,OK,[]
7,2118088,347015,NaN,"vejlgÃ¥rdvej 26, rÃ¸nbjerg, 7800 skive","VejlgÃ¥rdvej 26, 7800 Skive",100,"VelgÃ¥rdevej 26, 8766 NÃ¸rre Snede",65,"SvejgÃ¥rdsvej 26, 2900 Hellerup",65,OK,[]
8,2118090,347016,NaN,"lÃ¦rkegÃ¥rdsvÃ¦nget 5, ordrup, 4540 fÃ¥revejle","LÃ¦rkegÃ¥rdsvÃ¦nget 5, 4540 FÃ¥revejle",100,"VÃ¦nget 5, 2630 Taastrup",65,"VÃ¦nget 5, 3050 HumlebÃ¦k",65,WRONG,"['tiladre', 'tiladrn']"
9,2118092,347017,NaN,"hirsevej 17, 5800 nyborg","Hirsevej 17, 5800 Nyborg",100,"Hirsevej 17, 2700 BrÃ¸nshÃ¸j",80,"Hirsevej 17, 4040 Jyllinge",80,OK,[]
